In [1]:
import jsonlines
from bunkatopics.datamodel import Document, Term

from bunkatopics.datamodel import TopicRanking, BourdieuDimension, Term
from pydantic import BaseModel, Field
import typing as t


DOC_ID = str
TERM_ID = str
TOPIC_ID = str


class Document(BaseModel):
    doc_id: DOC_ID
    content: str
    size: t.Optional[float] = None
    x: t.Optional[float] = None
    y: t.Optional[float] = None
    topic_id: t.Optional[TOPIC_ID] = None
    topic_ranking: t.Optional[TopicRanking] = None  # Make topic_ranking optional
    term_id: t.Optional[t.List[TERM_ID]] = None
    embedding: t.Optional[t.List[float]] = Field(None, repr=False)
    bourdieu_dimensions: t.List[BourdieuDimension] = []



# Define a function to read documents from a JSONL file
def read_documents_from_jsonl(file_path):
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for item in reader:
            document = Document(**item)
            documents.append(document)
    return documents

In [2]:
documents = read_documents_from_jsonl("exports/bunka_docs_lemonde.jsonl")


In [3]:
import pandas as pd
df_embedding = pd.DataFrame([x.model_dump() for x in documents])
df_embedding = df_embedding[['doc_id', 'embedding']]

In [11]:
df_topics = pd.read_csv('exports/df_topics_top_docs.csv', index_col=[0])
df_topics['short_name'] = df_topics['topic_name'].apply(lambda x: '-'.join(x.split(' | ')[:7]))

In [12]:
len(df_topics.topic_id.value_counts())

25

In [18]:
df_topics_original = pd.read_csv('exports/topics.csv', index_col=[0])
list_topics = list(df_topics_original['topic_id'])

df_topics = df_topics[df_topics['topic_id'].isin(list_topics)]
df_topics

,doc_id,content,ranking_per_topic,topic_id,topic_name,short_name
0,07793ae8-4672-4edb-8,"""La Révolution française. Une histoire toujour...",1,bt-0,Révolution | COMBAT | démocratie | génération ...,Révolution-COMBAT-démocratie-génération-histoi...
1,376c20d0-3701-4b7e-a,"MILAN KUNDERA : "" En vidant une nation de sa c...",2,bt-0,Révolution | COMBAT | démocratie | génération ...,Révolution-COMBAT-démocratie-génération-histoi...
2,54b6797a-5c87-40f9-a,Accompagner les jeunes d'aujourd'hui et de dem...,3,bt-0,Révolution | COMBAT | démocratie | génération ...,Révolution-COMBAT-démocratie-génération-histoi...
3,60c7f95e-a30d-4df9-9,Les réactions de la presse À PARIS LA CROIX : ...,4,bt-0,Révolution | COMBAT | démocratie | génération ...,Révolution-COMBAT-démocratie-génération-histoi...
4,7de09df4-27d9-47f3-9,Vives réactions de la presse,5,bt-0,Révolution | COMBAT | démocratie | génération ...,Révolution-COMBAT-démocratie-génération-histoi...
...,...,...,...,...,...,...
20483,c9bfda11-e15d-42ec-a,LE PREMIER MINISTRE MALAIS VIENT DISCUTER DU P...,1996,bt-9,GÉNÉRAL | PRÉSIDENT | DÉMISSION | PRÉSIDENCE |...,GÉNÉRAL-PRÉSIDENT-DÉMISSION-PRÉSIDENCE-ÉLU-GAU...
20484,c9e97d11-05c5-4e98-b,M. RENÉ ABJEAN NOMMÉ DIRECTEUR DE RADIO-FINISTÈRE,1997,bt-9,GÉNÉRAL | PRÉSIDENT | DÉMISSION | PRÉSIDENCE |...,GÉNÉRAL-PRÉSIDENT-DÉMISSION-PRÉSIDENCE-ÉLU-GAU...
20485,c9edf2b0-7d46-4ab6-a,M. Moutet a reçu M. Ho Chi Minh,1998,bt-9,GÉNÉRAL | PRÉSIDENT | DÉMISSION | PRÉSIDENCE |...,GÉNÉRAL-PRÉSIDENT-DÉMISSION-PRÉSIDENCE-ÉLU-GAU...
20486,ca3fe13b-24d6-453e-a,"Dassault à la tête de la Socpresse, une satisf...",1999,bt-9,GÉNÉRAL | PRÉSIDENT | DÉMISSION | PRÉSIDENCE |...,GÉNÉRAL-PRÉSIDENT-DÉMISSION-PRÉSIDENCE-ÉLU-GAU...


In [19]:
df_final = pd.merge(df_embedding, df_topics, on = 'doc_id')

In [20]:
df_final.head(3)

,doc_id,embedding,content,ranking_per_topic,topic_id,topic_name,short_name
0,f9d2e3e9-81b5-4072-a,"[-0.03271692246198654, 0.06378008425235748, 0....","Discrètement, le gouvernement a prévu de relev...",810,bt-15,entreprises | logement | emploi | crise | chôm...,entreprises-logement-emploi-crise-chômage-entr...
1,7c5d0504-0854-4f2d-8,"[0.026198111474514008, 0.0498882457613945, -0....",GUIDE,161,bt-20,SEMAINE | GUIDE | week | GALERIES | end | TRAV...,SEMAINE-GUIDE-week-GALERIES-end-TRAVERS-entrées
2,8a73a0d0-296e-4de4-a,"[-0.05174738168716431, 0.0240201186388731, 0.0...",Le Sénat fait la chasse aux fraudeurs à la red...,997,bt-7,RÉFORME | LOI | ASSEMBLÉE | SYNDICATS | PROJET...,RÉFORME-LOI-ASSEMBLÉE-SYNDICATS-PROJET-députés...


In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder


In [22]:
# Convert embedding column to numpy array
X = np.array(df_final['embedding'].tolist())

# Convert topic_id to categorical labels
y = df_final['topic_id']

# Convert topic_id to integer labels using label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [23]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Create XGBoost classifier
model = XGBClassifier(objective='multi:softmax', num_class=len(set(y)))

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [33]:
import pickle

# Save the trained model to a file
with open('exports/model/xgboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the encoder to find back the topics
with open('exports/model/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [24]:
# Decode the predicted labels back to original topic IDs
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_pred_decoded

array(['bt-7', 'bt-6', 'bt-2', ..., 'bt-1', 'bt-3', 'bt-20'], dtype=object)

In [25]:
# Decode the predicted labels back to original topic IDs
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Classification report
target_names = label_encoder.classes_

topic_short_dict = df_final[['topic_id', 'short_name']].set_index('topic_id')['short_name'].to_dict()

target_names_short_names = [topic_short_dict[x] for x in target_names]

Accuracy: 0.8298538622129437


In [26]:

report = classification_report(y_test_decoded, y_pred_decoded, target_names=target_names_short_names)
print(report)

                                                                     precision    recall  f1-score   support

     Révolution-COMBAT-démocratie-génération-histoire-femmes-presse       0.80      0.58      0.68       139
FAIM-MANIFESTATIONS-migrants-COUR-milliers-MANIFESTATION-avortement       0.86      0.64      0.74       176
         ANS-Questions-ordinateur-siècle-III-TÉLÉVISION-principales       0.83      0.72      0.77       172
          MORTS-ACCIDENTS-INCENDIE-blessés-AVION-Airbus-inondations       0.93      0.82      0.87       199
          entreprises-logement-emploi-crise-chômage-entreprise-euro       0.74      0.71      0.73       245
     CONFÉRENCE-ALGÉRIE-visite-président-Calédonie-RELATIONS-traité       0.72      0.79      0.76       590
                 COUPE-TOUR-CHAMPIONNATS-MONDE-Mondial-ÉQUIPE-Bleus       0.91      0.93      0.92       288
            gauche-droite-ÉLECTIONS-Pen-SOCIALISTES-MAJORITÉ-Macron       0.75      0.81      0.78       613
                  